In [ ]:
# Importing pandas library
import pandas as pd
# Reading file 
star_wars_9_df = pd.read_csv("/content/INST737_Star_Wars_IMDB_StarWars9.csv")

In [ ]:
star_wars_9_df.head()

,Unnamed: 0,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,0,20 December 2019,BandSAboutMovies,4.0,Oh well\n,Three movies in to the final part of what is b...,/review/rw5331723/?ref_=tt_urv
1,1,21 December 2019,ben_busche,4.0,RIP Star Wars. Every inconsistency listed bel...,This movie introduced an absurd amount of inco...,/review/rw5331723/?ref_=tt_urv
2,2,2 February 2021,Prichards12345,1.0,Utter Garbage\n,"A hot mess of a movie, with terrible writing, ...",/review/rw5331723/?ref_=tt_urv
3,3,5 January 2020,danielkanemusic,2.0,The RISE (???) of Skywalker\n,The good:\nCinematography\nMusic,/review/rw5331723/?ref_=tt_urv
4,4,18 May 2020,arapacis-metal,NaN,Disney And JJ Abrams Completely Misunderstand...,They are so used to making films for four-year...,/review/rw5331723/?ref_=tt_urv


In [ ]:
# Removing unnecessary columns
star_wars_9_df_v2 = star_wars_9_df.drop(columns = ["Review_Date", "Author", "Rating", "Review_Title", "Review_Url", "Unnamed: 0"])

In [ ]:
star_wars_9_df_v2

,Review
0,Three movies in to the final part of what is b...
1,This movie introduced an absurd amount of inco...
2,"A hot mess of a movie, with terrible writing, ..."
3,The good:\nCinematography\nMusic
4,They are so used to making films for four-year...
...,...
8046,Alright now I get why hardcore fans are mad ab...
8047,"First of all, I am a big fan, but not a die-ha..."
8048,Very well written. Action packed. Good cinemat...
8049,I can't believe people are actually giving thi...


In [ ]:
# Topic Modeling Section


In [ ]:
# Installing Topic Modeling Libraries
!pip install regex
!pip install numpy
!pip install gensim
!pip install spacy
!pip install pyLDAvis
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=22a7391a05639492cc51c2c2ba8d51b6f693028dbe692b43bd84b99aabf0a514
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created whe

In [ ]:
# Importing Necessary Libraries
import regex
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import nltk
import regex as re
nltk.download('stopwords')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Removing emails and newline characters
data = star_wars_9_df_v2['Review'].tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", str(sent)) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-9-e93fef7f7c3a>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]
<ipython-input-9-e93fef7f7c3a>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', str(sent)) for sent in data]


Three movies in to the final part of what is being called the Skywalker Saga and I have to confess: worse than not knowing anything about these new characters -- Poe Dameron, Finn and Rey -- I honestly dont care about them. Theyre ciphers that we only know the barest of things about. Why does Finn have such an affinity for his friends? Why should we care a lick about Poe when hes proven to be too headstrong and willing to leave people behind, yet has nothing redeeming to make up for it? And Rey has every single skill of the Jedi near-instantly, almost a way too perfect character to be true.


In [ ]:
# Tokenize words and Clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['three', 'movies', 'in', 'to', 'the', 'final', 'part', 'of', 'what', 'is', 'being', 'called', 'the', 'skywalker', 'saga', 'and', 'have', 'to', 'confess', 'worse', 'than', 'not', 'knowing', 'anything', 'about', 'these', 'new', 'characters', 'poe', 'dameron', 'finn', 'and', 'rey', 'honestly', 'dont', 'care', 'about', 'them', 'theyre', 'ciphers', 'that', 'we', 'only', 'know', 'the', 'barest', 'of', 'things', 'about', 'why', 'does', 'finn', 'have', 'such', 'an', 'affinity', 'for', 'his', 'friends', 'why', 'should', 'we', 'care', 'lick', 'about', 'poe', 'when', 'hes', 'proven', 'to', 'be', 'too', 'headstrong', 'and', 'willing', 'to', 'leave', 'people', 'behind', 'yet', 'has', 'nothing', 'redeeming', 'to', 'make', 'up', 'for', 'it', 'and', 'rey', 'has', 'every', 'single', 'skill', 'of', 'the', 'jedi', 'near', 'instantly', 'almost', 'way', 'too', 'perfect', 'character', 'to', 'be', 'true']


In [ ]:
!python -m spacy download en_core_web_sm


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-11 00:38:57.823855: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['they', 'are', 'so', 'used', 'to', 'making', 'films', 'for', 'four', 'year_olds', 'that', 'they', 'thought', 'the', 'force', 'is', 'magic', 'there', 'are', 'magical', 'and', 'inexplicable', 'wizardry', 'happening', 'all', 'the', 'time', 'who', 'knew', 'jedi', 'can', 'heal', 'the', 'drying', 'and', 'mortally', 'wounded', 'all', 'those', 'jedi', 'didnt', 'need', 'to', 'die', 'in', 'episodes', 'iii', 'if', 'only', 'yoda', 'had', 'known', 'it', 'is', 'not', 'just', 'that', 'nurse', 'rey', 'is', 'so', 'magical', 'people', 'fly', 'people', 'transport', 'items', 'across', 'the', 'galaxy', 'people', 'come', 'back', 'from', 'the', 'dead', 'and', 'intervene', 'assist', 'interfere', 'and', 'the', 'list', 'continues']


In [ ]:
# Remove Stopwords and Lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

# nlp = spacy.load("en_core_web_sm").load()
# nlp = spacy.load('en', disable=['parser', 'ner'])
# python -m spacy download en_core_web_sm

# nlp = en_core_web_sm.load()
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['movie', 'final', 'part', 'call', 'skywalker', 'saga', 'confess', 'worse', 'know', 'new', 'character', 'honestly', 'care', 're', 'cipher', 'know', 'bare', 'thing', 'finn', 'affinity', 'friend', 'care', 'prove', 'headstrong', 'willing', 'leave', 'people', 'behind', 'yet', 'redeem', 'make', 'rey', 'instantly', 'almost', 'way', 'perfect', 'character', 'true']


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]


In [ ]:
!pip install -U gensim
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 20 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 91.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ld

[(0, '0.041*"child" + 0.036*"incredible" + 0.030*"impossible" + 0.027*"couple" + 0.027*"closure" + 0.024*"theatre" + 0.024*"young" + 0.020*"stay" + 0.020*"parent" + 0.018*"stick"'), (1, '0.040*"feeling" + 0.029*"dont_listen" + 0.027*"dialogue" + 0.027*"sit" + 0.025*"hater" + 0.024*"entertaining" + 0.021*"predictable" + 0.019*"base" + 0.019*"tear" + 0.015*"depth"'), (2, '0.072*"however" + 0.047*"away" + 0.036*"positive" + 0.022*"level" + 0.019*"meet" + 0.018*"soon" + 0.018*"sacrifice" + 0.014*"joy" + 0.014*"involve" + 0.013*"ago"'), (3, '0.042*"need" + 0.036*"always" + 0.032*"line" + 0.024*"half" + 0.024*"entire" + 0.023*"character" + 0.022*"honestly" + 0.022*"actor" + 0.021*"entertain" + 0.021*"main"'), (4, '0.061*"amazing" + 0.035*"work" + 0.032*"expectation" + 0.023*"fantastic" + 0.022*"special_effect" + 0.018*"score" + 0.018*"screen" + 0.017*"abrams" + 0.016*"deserve" + 0.014*"fitting"'), (5, '0.054*"epic" + 0.042*"battle" + 0.027*"future" + 0.024*"evil" + 0.022*"fast_pace" + 0.020*

In [ ]:
# Visualize the models
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.477274  0.045151       1        1  60.888157
8     -0.065448 -0.307979       2        1   4.678804
6     -0.018041 -0.166023       3        1   4.547342
3     -0.048259  0.253041       4        1   4.453186
4      0.010472  0.011986       5        1   3.819907
13     0.006575  0.011289       6        1   3.484141
12     0.040185  0.012861       7        1   2.860207
11     0.032537  0.010426       8        1   2.825573
1      0.061320  0.016980       9        1   2.185839
5      0.068451  0.015289      10        1   2.046175
10     0.073506  0.036820      11        1   1.731811
9      0.072082  0.014655      12        1   1.721059
2      0.077869  0.014886      13        1   1.685760
14     0.084391  0.015902      14        1   1.667814
0      0.081635  0.014717      15        1   1.404223, topic_info=           Term         Freq        Total Category  logprob  loglift
28    skywalker  1490.000000  1490.000000  Default  30.0000  30.0000
18        movie  9688.000000  9688.000000  Default  29.0000  29.0000
1160      enjoy  1133.000000  1133.000000  Default  28.0000  28.0000
168        rise   933.000000   933.000000  Default  27.0000  27.0000
120       scene  1021.000000  1021.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1322     homage    22.063305    22.926288  Topic15  -5.0760   4.2273
2692      setup    21.415163    22.278147  Topic15  -5.1059   4.2262
3358     bottom    21.042543    21.905528  Topic15  -5.1234   4.2255
3636      toxic    20.751231    21.614214  Topic15  -5.1373   4.2249
859       heavy    20.590464    21.453447  Topic15  -5.1451   4.2246

[488 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3867     15  0.969054    abram
885       5  0.996729   abrams
425       4  0.994936    actor
178       4  0.996983      add
199       8  0.964127    admit
...     ...       ...      ...
1047      5  0.998690     work
4907      8  0.964604  worried
1048     10  0.978923    worst
272       6  0.996656    wrong
1156     15  0.986668    young

[464 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 7, 4, 5, 14, 13, 12, 2, 6, 11, 10, 3, 15, 1])